In [ ]:
!pip install -q rasterio

In [ ]:
# Bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import reshape_as_raster
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Lendo imagem
with rio.open('/content/drive/MyDrive/Curso PDI com Python/L71221071_07120010720_DN.tif') as src:
  meta = src.meta
  img = reshape_as_image(src.read())

In [ ]:
# Lendo classes
with rio.open('roi.tif') as src2:
  roi = src2.read(1)

In [ ]:
# Armazenando metadados
meta = src.profile

In [ ]:
# Criando conjunto de treino
X = img[roi > 0]
y  = roi[roi > 0]

In [ ]:
# Dividindo conjunto em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(),
                                                    test_size = 0.3)

In [ ]:
# Instanciando o classificador
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(25, 12), random_state=1)

mlp.fit(X_train, y_train)

In [ ]:
# Criando matriz com bandas em colunas
img2d = img.reshape(img.shape[0]*img.shape[1], img.shape[2])

# Rodando predição
pred_mlp = mlp.predict(img2d)

# Retornando para o formato original
pred_mlp_final = pred_mlp.reshape(img[:,:,0].shape)

In [ ]:
# Visualizando resultado
plt.figure(figsize=(12,8))
plt.imshow(pred_mlp_final, cmap='Spectral')

In [ ]:
# Gravando em disco
meta.update({'count': 1})
with rio.open('pred_mlp_final.tif', 'w', **meta) as src:
    src.write(pred_mlp_final,1)